In [ ]:
import sys
sys.path.append('..')

In [2]:
import aequilibrae
import os
from os.path import join
import sys
import numpy as np
import pandas as pd
import openmatrix as omx
from aequilibrae import Project
from aequilibrae.paths import TrafficAssignment
from aequilibrae.paths import TrafficClass
from aequilibrae.matrix import AequilibraeMatrix, AequilibraeData

..\aequilibrae\parameters.py:51: UserWarning: No pre-existing parameter file exists for this project. Will use default
  warn('No pre-existing parameter file exists for this project. Will use default')
..\aequilibrae\parameters.py:51: UserWarning: No pre-existing parameter file exists for this project. Will use default
  warn('No pre-existing parameter file exists for this project. Will use default')


In [3]:
proj_folder = 'D:/release/Sample models/Chicago_2020_02_15/1_project'
proj = Project()
proj.open(proj_folder)

In [4]:
from aequilibrae.paths import PathResults

In [8]:
network = proj.network
network.build_graphs()

res = PathResults()
car_graph = network.graphs['c']
car_graph.set_graph('distance')
res.prepare(car_graph)


In [21]:
paths = []
for j in range(1, 1789):
    res.compute_path(j, 1)
    for i in range(1, 1789):
        paths.append(res.path.shape[0])
        res.update_trace(i)

In [23]:
max(paths)

237

In [ ]:
network = proj.network
network.build_graphs()

In [ ]:
mat_car = AequilibraeMatrix()
mat_car.load(join(proj_folder, 'matrices','MatODPico.aem'))
mat_car.computational_view(['carro'])

In [ ]:
mat_moto = AequilibraeMatrix()
mat_moto.load(join(proj_folder, 'matrices','MatODPico.aem'))
mat_moto.computational_view(['moto'])

In [ ]:
car = TrafficClass(network.graphs['c'],mat_car)
car.set_pce(1.0)
moto = TrafficClass(network.graphs['m'],mat_moto)
moto.set_pce(0.5)

In [ ]:
assig = TrafficAssignment()
assig.set_classes([car, moto])
assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": 0.15, "beta": 4.0}) # And its parameters
assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("FreeFlowTime")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

In [ ]:
volumes = assig.results()

In [ ]:
volumes.head()

In [ ]:
assig.assignment.convergence_report

In [ ]:
assig.save_results('tutorial_ime')